<a href="https://colab.research.google.com/github/PranavMunigala/Bioinformatics/blob/main/IntroToBioinformaticsColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Why We Do This**: The whole point of this process is to sift through tons of bioactivity data to find the compounds that might be worth exploring further as potential drugs. By pulling data from big databases and cleaning it up, we can zero in on the compounds that show real promise. This makes it easier to focus on the ones that are most likely to work, which can save a lot of time and effort in drug development and increase the chances of finding effective treatments.

Key vocabulary:

*   **Bioactivity**: The effect a compound has on a biological target, such as a protein.

* **Standard Value**: A numerical measure of a compound's bioactivity, which can be represented in various units like IC50 or EC50.


*   **EC50** (Half-Maximal Effective Concentration): The concentration required to achieve 50% of the maximum effect.


*   **Canonical SMILES**: A text-based notation for representing chemical structures.

*   **Data Pre-Processing**: The process of cleaning and organizing raw data to prepare it for analysis.



#Step 1: Install necessary libraries
In Google Colab, you'll first need to install the chembl_webresource_client package to retrieve bioactivity data from the ChEMBL Database.

In [ ]:
# Install the ChEMBL web service client
!pip install chembl_webresource_client

#Step 2: Import necessary libraries
 Once the package is installed, import the required libraries.

In [ ]:
# Import libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

# Step 3: Search for target protein (Acetylcholinesterase)
Now, let's search for the target protein, Acetylcholinesterase.

In [ ]:
# Target search for Acetylcholinesterase
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
targets

#Step 4: Retrieve Bioactivity Data for EC50
 "EC50" as the standard type (limit 800 entries)

In [ ]:
# Select the first entry, which corresponds to Human Acetylcholinesterase
selected_target = targets.target_chembl_id[0]

# Retrieve only the first 800 bioactivity data points with standard type 'EC50'
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="EC50")[:800]

# Convert the results into a DataFrame
df = pd.DataFrame.from_dict(res)

# Display the first few rows of the data
df.head()


#Step 5: Handle missing data
Filter out rows where there are missing values for **standard_value** and **canonical_smiles.**

In [ ]:
# Drop rows with missing standard_value or canonical_smiles
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]

# Display the first few rows after handling missing data
df2.head()

#Step 6: Remove duplicate entries
Ensure that each compound (represented by **canonical_smiles**) appears only once.

In [ ]:
# Remove duplicate canonical_smiles entries
df2_nr = df2.drop_duplicates(['canonical_smiles'])

# Display the first few rows of the non-redundant data
df2_nr.head()

#Step 7: Data Pre-processing
Select relevant columns and prepare a DataFrame.

In [ ]:
# Select columns for pre-processing
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2_nr[selection]

# Display the pre-processed data
df3.head()

#Step 8: Label compounds as active, inactive, or intermediate
Label compounds based on their Standard values into three categories: active, inactive, or intermediate.

In [ ]:

# Define the bioactivity classification
bioactivity_threshold = []
for value in df3.standard_value:
    if pd.isna(value):
        bioactivity_threshold.append("unknown")
    elif value >= 10000:
        bioactivity_threshold.append("inactive")
    elif value <= 1000:
        bioactivity_threshold.append("active")
    else:
        bioactivity_threshold.append("intermediate")

# Add the classification to the DataFrame
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df3, bioactivity_class], axis=1)

# Display the DataFrame with classifications
df5.head()


#Step 9: Save the processed data to a CSV file



You can save the final DataFrame to a CSV file.

In [ ]:
# Save the processed data to a CSV file
df5.to_csv('acetylcholinesterase_bioactivity_data_curated.csv', index=False)

#Step 10: Display the final DataFrame
Finally, you can display the entire processed DataFrame.

In [ ]:
# Display the entire DataFrame (may want to use df5.head() for large datasets)
df5

#Step 11 (Optional): Download the CSV file from Google Colab
If you want to download the CSV file directly from the Colab environment, you can use the following command:

In [ ]:
from google.colab import files
files.download('acetylcholinesterase_bioactivity_data_curated.csv')